In [ ]:
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import multiprocessing
from gensim.models import Word2Vec
from tqdm import tqdm

In [5]:
class CorpusIterator:
    """
    Parquet 파일을 배치 단위로 읽어 메모리 효율적으로 말뭉치를 생성하는 이터레이터.
    """
    def __init__(self, file_paths, batch_size=100_000, column='seq'):
        self.file_paths = file_paths
        self.batch_size = batch_size
        self.column = column

    def __iter__(self):
        for file_path in self.file_paths:
            print(f"Streaming from {file_path}...")
            # Parquet 파일을 열고 배치 단위로 순회
            parquet_file = pq.ParquetFile(file_path)
            for batch in parquet_file.iter_batches(batch_size=self.batch_size, columns=[self.column]):
                # Arrow Batch를 Pandas Series로 변환
                series = batch.to_pandas()[self.column]
                # 각 행의 문자열을 쉼표로 분리
                for sentence in series.str.split(','):
                    # 비어있는 요소 '' 제거 후 문장(리스트)을 반환(yield)
                    # sentence가 None일 경우를 대비하여 빈 리스트로 처리
                    yield [word for word in (sentence or []) if word]

# --- 최종 실행 ---
# 처리할 파일 경로 리스트
# file_paths = ['./data/train_optimized.parquet', './data/test_optimized.parquet']
file_paths = ['./data/train_optimized.parquet']

# 이터레이터 객체 생성
# 이 시점에는 아직 데이터를 읽지 않습니다.
corpus_iterator = CorpusIterator(file_paths)

# Word2Vec 모델 학습 시, corpus_iterator를 sentences 인자로 직접 전달
# 모델이 필요할 때마다 데이터를 조금씩 스트리밍하여 사용합니다.
VECTOR_SIZE = 64
WINDOW = 5
MIN_COUNT = 3
WORKERS = multiprocessing.cpu_count()
SG = 1

print("Training Word2Vec model with memory-efficient iterator...")
w2v_model = Word2Vec(
    sentences=corpus_iterator,  # <--- 메모리에 모든 데이터를 올리는 대신 이터레이터 사용!
    vector_size=VECTOR_SIZE,
    window=WINDOW,
    min_count=MIN_COUNT,
    workers=WORKERS,
    sg=SG,
    epochs=5
)

print("Model training complete.")
w2v_model.save("./models/item2vec.model")

Training Word2Vec model with memory-efficient iterator...
Streaming from ./data/train_optimized.parquet...
Streaming from ./data/train_optimized.parquet...
Streaming from ./data/train_optimized.parquet...
Streaming from ./data/train_optimized.parquet...
Streaming from ./data/train_optimized.parquet...
Streaming from ./data/train_optimized.parquet...
Model training complete.


In [ ]:
def get_sentence_vector(sentence, model, vector_size):
    """문장(ID 리스트)을 벡터화 (numpy 연산)"""
    words = [w for w in sentence if w in model.wv]
    if not words:
        return np.zeros(vector_size, dtype=np.float32)
    arr = model.wv[words]  # (len(words), vector_size) numpy 배열
    return arr.mean(axis=0)

def build_seq_vectors(df, model, vector_size, col="seq", batch_size=100_000):
    """대규모 데이터를 배치 단위로 변환"""
    vec_list = []
    total = len(df)
    
    for start in tqdm(range(0, total, batch_size)):
        end = min(start + batch_size, total)
        batch = df[col].iloc[start:end].str.split(",")
        batch_vecs = np.vstack([
            get_sentence_vector(seq, model, vector_size) for seq in batch
        ])
        vec_list.append(batch_vecs)

    all_vecs = np.vstack(vec_list)

    return all_vecs
    

train_seq = pd.read_parquet('./data/train_optimized.parquet', columns=['seq'])
test_seq = pd.read_parquet('./data/test_optimized.parquet', columns=['seq'])

# 실행
print("Creating sequence vectors for train and test data...")

train_seq_vectors = build_seq_vectors(
    train_seq, w2v_model, VECTOR_SIZE, col="seq",
)

test_seq_vectors = build_seq_vectors(
    test_seq, w2v_model, VECTOR_SIZE, col="seq",
)

print("Sequence vectors created.")


In [ ]:
# 이전에 만들어둔 베이스라인 데이터셋을 불러옵니다.
# (예: train_baseline.parquet)
train_baseline_df = pd.read_parquet('./data/train_optimized.parquet') 
test_baseline_df = pd.read_parquet('./data/test_optimized.parquet')

train_seq_stats = pd.read_parquet('./data/processed/train_seq_stats.parquet')
test_seq_stats = pd.read_parquet('./data/processed/test_seq_stats.parquet')

train_seq_vec_df = pd.DataFrame(train_seq_vectors, columns=[f"seq_vec_{i}" for i in range(VECTOR_SIZE)])
test_seq_vec_df = pd.DataFrame(test_seq_vectors, columns=[f"seq_vec_{i}" for i in range(VECTOR_SIZE)])

seq_cols = ['seq_len', 'seq_first', 'seq_last']

# 벡터 피처들을 옆으로 이어 붙입니다.
train_final = pd.concat([train_baseline_df, train_seq_stats[seq_cols], train_seq_vec_df], axis=1)
test_final = pd.concat([test_baseline_df, test_seq_stats[seq_cols], test_seq_vec_df], axis=1)

print("Final dataset is ready.")
print("New train shape:", train_final.shape)